<a href="https://colab.research.google.com/github/george-zakharov/ml-random/blob/master/lips_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load libs

In [9]:
%tensorflow_version 2.x

In [10]:
import tensorflow
print(tensorflow.__version__)

2.3.0


In [35]:
import os
import shutil
import glob

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from google.colab import drive
from google.colab import data_table

# from google.colab import files
from tensorflow.keras.preprocessing import image
# from tensorflow.keras import utils
# from keras.preprocessing.image import ImageDataGenerator
# from keras.models import Model
# from keras.layers import Input, Conv2DTranspose, concatenate, Activation, MaxPooling2D, Conv2D, BatchNormalization
# from keras import backend as K
# from keras.optimizers import Adam
# import matplotlib.pyplot as plt

# Load data

In [ ]:
drive.mount('/content/drive', force_remount=True)

In [45]:
# Get raw data
!cp '/content/drive/My Drive/Colab Notebooks/segmentation_test/lips.zip' lips.zip

In [4]:
# Get pretrained values
!cp '/content/drive/My Drive/Colab Notebooks/segmentation_test/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5' vgg16.h5

In [ ]:
# Unzip it
!unzip lips.zip

In [47]:
# Delete archive
!rm lips.zip

# Check the mappings

In [7]:
mappings = pd.read_csv('set-lipstick-original/list.csv')

In [37]:
mappings.head()

,filename,width,height,class,xmin,ymin,xmax,ymax,mask
0,image00000001.jpg,1280,720,Lips,661,394,776,444,mask00000001.png
1,image00000002.jpg,1280,720,Lips,557,336,682,392,mask00000002.png
2,image00000003.jpg,1280,720,Lips,553,369,684,427,mask00000003.png
3,image00000004.jpg,1280,720,Lips,555,351,681,408,mask00000004.png
4,image00000005.jpg,1280,720,Lips,555,351,680,407,mask00000005.png


# Data processing

In [13]:
# Prepare dirs for files
!mkdir DIR_X_TRAIN
!mkdir DIR_X_TEST
!mkdir DIR_Y_TRAIN
!mkdir DIR_Y_TEST

In [48]:
# Get first 5000 pics for training
train_df = mappings.loc[0:4999]

In [49]:
# Get test pics
test_df = mappings.loc[5000:5099]

In [52]:
# Copy train pics
for index, row in train_df.iterrows():
    shutil.copy('/content/set-lipstick-original/720p/' + row['filename'], '/content/DIR_X_TRAIN/' + row['filename'])
    shutil.copy('/content/set-lipstick-original/mask/' + row['mask'], '/content/DIR_Y_TRAIN/' + row['mask'])

In [53]:
# Copy test pics
for index, row in test_df.iterrows():
    shutil.copy('/content/set-lipstick-original/720p/' + row['filename'], '/content/DIR_X_TEST/' + row['filename'])
    shutil.copy('/content/set-lipstick-original/mask/' + row['mask'], '/content/DIR_Y_TEST/' + row['mask'])

In [54]:
# Let's drop source images
!rm -rf /content/set-lipstick-original/

In [55]:
# Get all paths
x_test_files = glob.glob('/content/DIR_X_TEST/*')
x_train_files = glob.glob('/content/DIR_X_TRAIN/*')
y_test_files = glob.glob('/content/DIR_Y_TEST/*')
y_train_files = glob.glob('/content/DIR_Y_TRAIN/*')

# Create train data

In [60]:
# Create x_train
img = image.load_img(x_train_files[0], target_size=(352, 480))  
x_train = image.img_to_array(img)
x_train = np.expand_dims(x_train, axis=0)
x_train.shape

(1, 352, 480, 3)

In [ ]:
# Enrich x_train
for i in range(len(x_train_files)):
    if i > 0 : 
        img = image.load_img(x_train_files[i], target_size=(352,480))
        buf = image.img_to_array(img)
        buf = np.expand_dims(buf, axis=0)
        x_train = np.concatenate((x_train, buf), axis=0)
x_train.shape

In [ ]:
# Create ypic3_train
img = image.load_img(y_test_files[0], target_size=(352,480))  
ypic3_train = image.img_to_array(img)
ypic3_train = np.expand_dims(ypic3_train, axis=0)
ypic3_train.shape

In [ ]:
# Enrich ypic3_train
for i in range(len(y_test_files)):
    if i > 0 : 
        img = image.load_img(y_test_files[i], target_size=(352,480))
        buf = image.img_to_array(img)
        buf = np.expand_dims(buf, axis=0)
        ypic3_train = np.concatenate((ypic3_train, buf), axis=0)
ypic3_train.shape

In [ ]:
# Get y_train
y_train = np.zeros((367, 352, 480))

for num in range(367):
     for y in range(352):
          for x in range(480):
                cl = ypic3_train[num,y,x,0]
                if ypic3_train[num,y,x,1] == cl and ypic3_train[num,y,x,2] == cl:
                  y_train[num,y,x] = cl 
                else:
                  print('ERROR')

In [ ]:
for num in range(367):
     for y in range(352):
          for x in range(480):
              cl = y_train[num,y,x]
              if cl == 6 or cl == 7 or cl == 9 or cl == 10:
                 cl = 2
              if cl == 8:    
                 cl = 6
              if cl == 11:
                 cl = 7
              y_train[num, y, x] = cl 

In [ ]:
y_train = utils.to_categorical(y_train, 8)

In [ ]:
def dice_coef(y_true, y_pred):
    return (2. * K.sum(y_true * y_pred) + 1.) / (K.sum(y_true) + K.sum(y_pred) + 1.)


def unet(num_classes = 8, input_shape= (352, 480, 3)):
    img_input = Input(input_shape)

    # Block 1
    x = Conv2D(64, (3, 3), padding='same', name='block1_conv1')(img_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(64, (3, 3), padding='same', name='block1_conv2')(x)
    x = BatchNormalization()(x)
    block_1_out = Activation('relu')(x)

    x = MaxPooling2D()(block_1_out)

    # Block 2
    x = Conv2D(128, (3, 3), padding='same', name='block2_conv1')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(128, (3, 3), padding='same', name='block2_conv2')(x)
    x = BatchNormalization()(x)
    block_2_out = Activation('relu')(x)

    x = MaxPooling2D()(block_2_out)

    # Block 3
    x = Conv2D(256, (3, 3), padding='same', name='block3_conv1')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(256, (3, 3), padding='same', name='block3_conv2')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(256, (3, 3), padding='same', name='block3_conv3')(x)
    x = BatchNormalization()(x)
    block_3_out = Activation('relu')(x)

    x = MaxPooling2D()(block_3_out)

    # Block 4
    x = Conv2D(512, (3, 3), padding='same', name='block4_conv1')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(512, (3, 3), padding='same', name='block4_conv2')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(512, (3, 3), padding='same', name='block4_conv3')(x)
    x = BatchNormalization()(x)
    block_4_out = Activation('relu')(x)

    x = MaxPooling2D()(block_4_out)

    # Block 5
    x = Conv2D(512, (3, 3), padding='same', name='block5_conv1')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(512, (3, 3), padding='same', name='block5_conv2')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(512, (3, 3), padding='same', name='block5_conv3')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)


    # Load pretrained weights.
    for_pretrained_weight = MaxPooling2D()(x)
    vgg16 = Model(img_input, for_pretrained_weight)
    vgg16.load_weights('drive/My Drive/Colab Notebooks/7.Сегментация и продвинутые операции со свёрткой/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5', by_name=True)

    # UP 1
    x = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = concatenate([x, block_4_out])
    x = Conv2D(512, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(512, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # UP 2
    x = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = concatenate([x, block_3_out])
    x = Conv2D(256, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(256, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # UP 3
    x = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = concatenate([x, block_2_out])
    x = Conv2D(128, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(128, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # UP 4
    x = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = concatenate([x, block_1_out])
    x = Conv2D(64, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(64, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(num_classes, (3, 3), activation='softmax', padding='same')(x)

    model = Model(img_input, x)
    model.compile(optimizer=Adam(),
                  loss='categorical_crossentropy',
                  metrics=[dice_coef])
    model.summary()
    return model